# Day-1 Expert Knowledge Worker
A question answering agent that is an expert knowledge worker

To be used by employees of Insurellm, an Insurance Tech company

The agent needs to be accurate and the solution should be low cost



> This project will use RAG( Retrieval Augmented Generation ) to ensure our question/answering assistant has high accuracy.

This first implementation will use a simple, brute force type of RAG.



In [ ]:
!pip install openai
!pip install gradio

In [ ]:
#imports

import os
import glob
import gradio as gr
from google.colab import userdata
from openai import OpenAI

ImportError: cannot import name 'GetCoreSchemaHandler' from 'pydantic' (/usr/local/lib/python3.11/dist-packages/pydantic/__init__.cpython-311-x86_64-linux-gnu.so)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# price is a factor for our company, so we're going to use  low cost model
openapi_api_key = userdata.get('openapi')
MODEL = "gpt-4o-mini"
openai = OpenAI(api_key=openapi_api_key)

In [ ]:
context = {}
employees = glob.glob("/content/drive/MyDrive/knowledge-base/employees/*")

for employee in employees:
  name = employee.split(" ")[-1][:-3]
  doc = ""
  with open(employee, "r") as f:
    doc = f.read()
  context[name] = doc

In [ ]:
context.keys()
context["Lancaster"]

"# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferenc

In [ ]:
products = glob.glob("/content/drive/MyDrive/knowledge-base/products/*")

for product in products:
  name = product.split("/")[-1][:-3]
  doc = ""
  with open(product, "r") as f:
    doc = f.read()
  context[name] = doc

In [ ]:
context.keys()

dict_keys(['Thomson', 'Greene', 'Lancaster', 'Spencer', 'Trenton', 'Tran', 'Thompson', 'Bishop', 'Carter', 'Blake', 'Chen', 'Harper', '/content/drive/MyDrive/knowledge-base/products/Rellm', '/content/drive/MyDrive/knowledge-base/products/Markellm', '/content/drive/MyDrive/knowledge-base/products/Carllm', '/content/drive/MyDrive/knowledge-base/products/Homellm', 'Rellm', 'Markellm', 'Carllm', 'Homellm'])

In [ ]:
system_message = "You are an exprt in answering accurate questions about Insurellm, the Insurance Tech company. Give bried, accurate answer. if you don't know the answer, say so, Do not make anything up if you haven't been provided with relevent context"

In [ ]:
def get_relevent_context(message):
  relevent_context = []
  for context_title, context_details in context.items():
    if context_title in message:
      relevent_context.append(context_details)
  return relevent_context

In [ ]:
get_relevent_context("Who is Avery Lancaster and what is Carllm?")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

In [ ]:
def add_context(message):
  relevent_context = get_relevent_context(message)
  if relevent_context:
    message += "\n\n The following additional context might be relevent in answering the question: \n\n"
    for relevent in relevent_context:
      message += relevent + "\n\n"
  return message

In [ ]:
print(add_context("Who is Avery Lancaster?"))

Who is Avery Lancaster?

 The following additional context might be relevent in answering the question: 

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  
  Prior to joining Inno

In [ ]:
def chat(message, history):
  messages = [{"role": "system", "content": system_message}]
  for user, assistant in history:
    messages.append({"role": "user", "content": user})
    messages.append({"role": "assistant", "content": assistant})

  message = add_context(message)
  messages.append({"role": "user", "content": message})
  stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
  response = ""
  for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    yield response

In [ ]:
view = gr.ChatInterface(chat).launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3480c984645041a1b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Day-2 and Day-3 and Day-4 : Using Lanchain and Vectors and Chroma and RAG for above.


In [2]:
!pip install pydantic
!pip install numpy

In [3]:
!pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00


In [4]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [5]:
# imports
import glob
import gradio as gr
from openai import OpenAI
from google.colab import userdata
from google.colab import drive
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

#imports for RAG
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [6]:
# price is a factor for our company, so we're going to use  low cost model
drive.mount('/content/drive', force_remount=True)
openapi_api_key = userdata.get('openapi')
MODEL = "gpt-4o-mini"
openai = OpenAI(api_key=openapi_api_key)
db_name = "vector_db"

Mounted at /content/drive


In [35]:
# Read in documents using LangChain's loaders
#Take everything in all the sub folders of our knowledgebase
import os
folders = glob.glob("/content/drive/MyDrive/knowledge-base/*")

documents = []
for folder in folders:
  doc_type = os.path.basename(folder)
  print(doc_type, folder)
  loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader)
  folder_docs = loader.load()
  for doc in folder_docs:
    doc.metadata["doc_type"] = doc_type
    documents.extend(folder_docs)

products /content/drive/MyDrive/knowledge-base/products
contracts /content/drive/MyDrive/knowledge-base/contracts
company /content/drive/MyDrive/knowledge-base/company
employees /content/drive/MyDrive/knowledge-base/employees


In [37]:
documents

[Document(metadata={'source': '/content/drive/MyDrive/knowledge-base/products/Rellm.md', 'doc_type': 'products'}, page_content="# Product Summary\n\n# Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Summary\n\nRellm is an innovative enterprise reinsurance product developed by Insurellm, designed to transform the way reinsurance companies operate. Harnessing the power of artificial intelligence, Rellm offers an advanced platform that redefines risk management, enhances decision-making processes, and optimizes operational efficiencies within the reinsurance industry. With seamless integrations and robust analytics, Rellm enables insurers to proactively manage their portfolios and respond to market dynamics with agility.\n\n## Features\n\n### AI-Driven Analytics\nRellm utilizes cutting-edge AI algorithms to provide predictive insights into risk exposures, enabling users to forecast trends and make informed decisions. Its real-time data analysis empowers reinsurance professionals w

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
len(chunks)

1281

In [ ]:
chunks[6]

Document(metadata={'source': '/content/drive/MyDrive/knowledge-base/products/Markellm.md', 'doc_type': 'products'}, page_content='- **User-Friendly Interface**: Designed with user experience in mind, Markellm features an intuitive interface that allows consumers to easily browse and compare various insurance offerings from multiple providers.\n\n- **Real-Time Quotes**: Consumers can receive real-time quotes from different insurance companies, empowering them to make informed decisions quickly without endless back-and-forth communication.\n\n- **Customized Recommendations**: Based on user profiles and preferences, Markellm provides personalized insurance recommendations, ensuring consumers find the right coverage at competitive rates.\n\n- **Secure Transactions**: Markellm prioritizes security, employing robust encryption methods to ensure that all transactions and data exchanges are safe and secure.\n\n- **Customer Support**: Our dedicated support team is always available to assist bot

In [9]:
doc_types = set(chunk.metadata["doc_type"] for chunk in chunks)
doc_types

{'company', 'contracts', 'employees', 'products'}

In [ ]:
for chunk in chunks:
  if 'Lancaster' in chunk.page_content:
    print(chunk)
    print(f"_____")

page_content='# About Insurellm

Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. It's first product was Markellm, the marketplace connecting consumers with insurance providers.
It rapidly expanded, adding new products and clients, reaching 200 emmployees by 2024 with 12 offices across the US.' metadata={'source': '/content/drive/MyDrive/knowledge-base/company/about.md', 'doc_type': 'company'}
_____
page_content='# About Insurellm

Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. It's first product was Markellm, the marketplace connecting consumers with insurance providers.
It rapidly expanded, adding new products and clients, reaching 200 emmployees by 2024 with 12 offices across the US.' metadata={'source': '/content/drive/MyDrive/knowledge-base/company/about.md', 'doc_type': 'company'}
_____
page_con

# A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, knwon as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input. It's different to all the other LLMs we'vw discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-Encoding LLms are often used for classification.

In [10]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 28.2 MB/s eta 0:00:

In [11]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00


In [12]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

embeddings = OpenAIEmbeddings(openai_api_key=openapi_api_key)

<ipython-input-12-e9b0b72c1f81>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openapi_api_key)


In [14]:
# CHeck if a Chroma Datastore already exists - If so, delete the collection to start from scratch
if os.path.exists(db_name):
  Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

<ipython-input-14-e1b6f87ca436>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()


In [15]:
#Create our Chroma vectorstore
vector_store = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vector_store._collection.count()} documents")

Vectorstore created with 1281 documents


In [16]:
# Get one vector and find how many dimensions it has

collection = vector_store._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"Embeddings have {dimensions:,} dimensions")

Embeddings have 1,536 dimensions


# Visualizing the vector store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.


In [17]:
# Prework

result = collection.get(include=["embeddings","documents",'metadatas'])
vector = np.array(result["embeddings"])
documents = result["documents"]
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue','green','red','orange'] [['products','employees','contracts','company'].index(t)] for t in doc_types]

In [18]:
# We humans find it easier to visualize things in 2D!
# Reduce the dimensionality of the vectors to 2d using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
two_dim = tsne.fit_transform(vector)

#Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=two_dim[:,0],
    y=two_dim[:,1],
    mode='markers',
    marker=dict(size=5,color=colors,opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t,d in zip(doc_types,documents)], # text showd only 100 characters
    hoverinfo="text"
    )])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20,b=10,l=10,t=40)
)

fig.show()

In [19]:
# Let's try 3D!
tsne = TSNE(n_components=3, random_state=42)
two_dim = tsne.fit_transform(vector)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=two_dim[:,0],
    y=two_dim[:,1],
    z=two_dim[:, 2],
    mode='markers',
    marker=dict(size=5,color=colors,opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t,d in zip(doc_types,documents)], # text only 100 characters
    hoverinfo="text"
    )])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20,b=10,l=10,t=40)
)

fig.show()

# Time to use LangChain to bring it all together


In [23]:
#create a new CHat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL, api_key=openapi_api_key)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vector_store.as_retriever()

# putting it together: set up the conversation cahin with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

<ipython-input-23-bb83c6930c19>:5: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [25]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Insurellm is an innovative insurance tech startup founded by Avery Lancaster in 2015, designed to disrupt the insurance industry with new products. Its first product, Markellm, is a marketplace that connects consumers with insurance providers. The company has rapidly expanded, offering four insurance software products and serving over 300 clients worldwide, with a workforce of 200 employees across 12 offices in the US by 2024.


In [28]:
def chat(message, history):
  result = conversation_chain.invoke({"question": message})
  return result["answer"]

In [29]:
view = gr.ChatInterface(chat).launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning:

The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6d2cac78e96421c96.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [30]:
# Let's investigate what gets sent behind the scenes

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL, api_key=openapi_api_key)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
retriever = vector_store.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "who received the prestigious IIOTY award in 2023?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
- **2022**: **Satisfactory**  
  Avery focused on rebuilding team dynamics and addressing employee concerns, leading to overall improvement despite a saturated market.  

- **2023**: **Exceeds Expectations**  
  Market leadership was regained with innovative approaches to personalized insurance solutions. Avery is now recognized in industry publications as a leading voice in Insurance Tech innovation.

- **2022**: **Satisfactory**  
  Avery focused on rebuilding team dynamics and addressing employee concerns, leading to overall improvement despite a saturated market.  

- **2023**: **Exceeds Expectations**  
  Market leadership was regained with 

If the answer is wrong for the prompt in RAG, then to troubleshoot it, we can change the way we are handling chunks in the vector store

By sending the full document as a chunk

Dividing the chunks into smaller fine-grained contents

Or by setting the condition as to how much content it needs to take, cause the more content or chunks you pass to the LLM model, the better response it gives.

In [39]:
llm = ChatOpenAI(temperature=0.7, model_name=MODEL, api_key=openapi_api_key)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# the retriever is an abstraction over the Vector Store that will be used during RAG; k is how many chunks to use

retriever = vector_store.as_retriever(search_kwargs={"k": 500})
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "who received the prestigious IIOTY award in 2023?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Streaming output truncated to the last 5000 lines.

## Other HR Notes
- Jordan K. Bishop has been an integral part of club initiatives, including the Insurellm Code Reviews and Feedback Group, providing peer support.
- Active participant in the company's Diversity and Inclusion committee, promoting a positive work culture.
- Jordan has expressed interest in professional development courses, particularly those focused on modern web technologies, which are being considered for sponsorship by Insurellm.
- Engaged in a 6-month performance improvement plan as of August 2023, focusing on skill development and consistent performance monitoring. 

Jordan K. Bishop is a valued member of the Insurellm family, exhibiting a commitment to growth and development despite recent challenges.

## Other HR Notes
- Jordan K. Bishop has been an integral part of club initiatives, including the Insurellm Code Reviews and Feedback Group, providing peer support.
- Active participant in the company's Diversity 

In [40]:
view = gr.ChatInterface(chat).launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning:

The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://731e92d2015d45cbab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
